In [1]:
import os
import numpy as np

from MolRep import MolRep
from MolRep.Utils.logger import Logger
from MolRep.Utils.config_from_dict import Config
from MolRep.Experiments.experiments import EndToEndExperiment

In [2]:
MODEL_CONFIG_DIR = '../MolRep/Configs' # Need to set! The directory of Model Configurations files, such as config_CMPNN.yml.
OUTPUT_DIR = '../Outputs/'


_CONFIG_BASE = 'config_'
_CONFIG_FILENAME = 'config_results.json'

_FOLDS = 5
MODEL_NAME = 'CMPNN'

# define your dataset name
DATASET_NAME = 'BBB'  
# define the PATH of your data. Could be CSV or SDF format.
DATASET_PATH = '../MolRep/Datasets/BBBP/BBBP.csv'
# define the column name of SMILES in your data
SMILES_COLUMN = 'smiles'
# the column names of TARGET NAME in your data. Must be a List.
TARGET_COLUMNS = ['p_np']
# define the task type. Classification or Regression
TASK_TYPE = 'Classification'
# define the metric type, such as auc or rmse
METRIC_TYPE = 'auc'
# define the split data type, such as random, stratified, scaffold. NOTE that stratified only applies to single property
SPLIT_TYPE = 'random'

In [3]:
dataset_config, dataset, model_configurations, model_selector, exp_path = MolRep.construct_dataset(
    dataset_name = DATASET_NAME,
    model_name = MODEL_NAME,
    dataset_path = DATASET_PATH,
    smiles_column = SMILES_COLUMN,
    target_columns = TARGET_COLUMNS,
    task_type = TASK_TYPE,
    metric_type = METRIC_TYPE,
    split_type = SPLIT_TYPE,
    inner_k = _FOLDS,
    config_dir = MODEL_CONFIG_DIR,
    output_dir=OUTPUT_DIR
)

In [4]:
config_id = 0   # the idx of model configs since there are more than 100 combinations of hyper-parameters.
KFOLD_FOLDER = os.path.join(exp_path, str(_FOLDS) + '_FOLD_MS')
exp_config_name = os.path.join(KFOLD_FOLDER, _CONFIG_BASE + str(config_id + 1))
config_filename = os.path.join(exp_config_name, _CONFIG_FILENAME)
if not os.path.exists(exp_config_name):
    os.makedirs(exp_config_name)

In [5]:
config = model_configurations[config_id]

# model configs could be change
# for example:
# config['device'] = 'cpu' or config['batch_size'] = 32

logger = Logger(str(os.path.join(exp_config_name, 'experiment.log')), mode='w')
logger.log('Configuration: ' + str(config))

Configuration: {'model': 'CMPNN', 'device': 'cuda', 'batch_size': 50, 'learning_rate': 0.0001, 'l2': 0, 'hidden_size': 300, 'bias': False, 'depth': 3, 'dropout': 0.0, 'activation': 'ReLU', 'undirected': False, 'ffn_hidden_size': 256, 'ffn_num_layers': 2, 'atom_messages': False, 'no_cache': False, 'optimizer': 'Adam', 'scheduler': {'class': 'NoamLR', 'args': {'warmup_epochs': [2.0], 'step_size': 10, 'max_lr': [0.0001], 'init_lr': [1e-05], 'final_lr': [1e-05]}}, 'early_stopper': {'class': 'Patience', 'args': {'patience': 500, 'use_loss': False}}, 'gradient_clipping': None, 'num_epochs': 30, 'num_lrs': 1, 'features_only': False, 'atom_descriptors': 'None', 'use_input_features': False, 'shuffle': True}


In [6]:
k_fold_dict = {
    'config': config,
    'folds': [{} for _ in range(_FOLDS)],
    'avg_TR_score': 0.,
    'avg_VL_score': 0.,
    'std_TR_score': 0.,
    'std_VL_score': 0.
}

In [7]:
dataset_getter = MolRep.construct_dataloader(dataset)
for k in range(_FOLDS):
    logger.log(f"Training in Fold: {k+1}")
    dataset_getter.set_inner_k(k)

    fold_exp_folder = os.path.join(exp_config_name, 'FOLD_' + str(k + 1))
    # Create the experiment object which will be responsible for running a specific experiment
    experiment = EndToEndExperiment(config, dataset_config, fold_exp_folder)

    model_path = os.path.join(fold_exp_folder, f"{MODEL_NAME}_{DATASET_NAME}_fold_{k}.pt")
    training_score, validation_score = experiment.run_valid(dataset_getter, logger, other={'model_path': model_path})

    # print('training_score:', training_score, 'validation_score:',validation_score)
    logger.log(str(k+1) + ' split, TR Score: ' + str(training_score) +
                ' VL Score: ' + str(validation_score))

    k_fold_dict['folds'][k]['TR_score'] = training_score
    k_fold_dict['folds'][k]['VL_score'] = validation_score

tr_scores = np.array([k_fold_dict['folds'][k]['TR_score'] for k in range(_FOLDS)])
vl_scores = np.array([k_fold_dict['folds'][k]['VL_score'] for k in range(_FOLDS)])

k_fold_dict['avg_TR_score'] = tr_scores.mean()
k_fold_dict['std_TR_score'] = tr_scores.std()
k_fold_dict['avg_VL_score'] = vl_scores.mean()
k_fold_dict['std_VL_score'] = vl_scores.std()

log_str = f"TR avg is %.4f std is %.4f; VL avg is %.4f std is %.4f" % (
            k_fold_dict['avg_TR_score'], k_fold_dict['std_TR_score'], k_fold_dict['avg_VL_score'], k_fold_dict['std_VL_score']
        )
logger.log(log_str)

Training in Fold: 1
[TRAIN] Epoch: 1, train loss: 0.603535 train auc: 0.519052
[VALID] Epoch: 1, valid loss: 0.574165 valid auc: 0.590566
- Elapsed time: 8.67s , Time estimation in a fold: 4.33min
[TRAIN] Epoch: 10, train loss: 0.338314 train auc: 0.891489
[VALID] Epoch: 10, valid loss: 0.511618 valid auc: 0.864862
- Elapsed time: 2.39s , Time estimation in a fold: 1.19min
[TRAIN] Epoch: 20, train loss: 0.217411 train auc: 0.956670
[VALID] Epoch: 20, valid loss: 0.479814 valid auc: 0.924324
- Elapsed time: 2.30s , Time estimation in a fold: 1.15min
[TRAIN] Epoch: 30, train loss: 0.147633 train auc: 0.980445
[VALID] Epoch: 30, valid loss: 0.465639 valid auc: 0.948089
- Elapsed time: 2.58s , Time estimation in a fold: 1.29min
1 split, TR Score: 0.9804451175901384 VL Score: 0.9480893073126082
Training in Fold: 2
[TRAIN] Epoch: 1, train loss: 0.591510 train auc: 0.503150
[VALID] Epoch: 1, valid loss: 0.549290 valid auc: 0.680348
- Elapsed time: 2.65s , Time estimation in a fold: 1.32min
[T